In [12]:
#import dataset

import tensorflow as tf 
import tensorflow_datasets as tfds
import io

In [13]:
#download data set
#imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

#load data setnya yang sudah didownload manual
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True, data_dir="./data/", download=False)



In [14]:
#tampilkan informasi tentang dataset
print(info)

tfds.core.DatasetInfo(
    name='imdb_reviews',
    full_name='imdb_reviews/plain_text/1.0.0',
    description="""
    Large Movie Review Dataset. This is a dataset for binary sentiment
    classification containing substantially more data than previous benchmark
    datasets. We provide a set of 25,000 highly polar movie reviews for training,
    and 25,000 for testing. There is additional unlabeled data for use as well.
    """,
    config_description="""
    Plain text
    """,
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    data_dir='data\\imdb_reviews\\plain_text\\1.0.0',
    file_format=tfrecord,
    download_size=80.23 MiB,
    dataset_size=129.83 MiB,
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
        'text': Text(shape=(), dtype=string),
    }),
    supervised_keys=('text', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=25000, num_shards=1>,
        'train': <Spli

In [15]:
print(imdb)

{'train': <_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>, 'test': <_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>, 'unsupervised': <_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>}


### Preview Data

In [16]:
#with_info=True : menerima informasi tambahan dari dataset
#as_supervised=True : konversi data ke bentuk yang lebih sederhana (input, label) agar mudah digunakan dalam supervised learning
#download=False berarti kita tidak perlu mengunduh ulang dataset ini karena sudah tersedia secara lokal.

#melihat satu contoh ulasan
single_example=list(imdb['train'].take(1))[0]

#take(1) : mengambil 1 contoh 
# list()[0] : konversi jadi list dan ambil item pertama

#tampilkan hasil
print(single_example[0]) #kalau atasnya 0 berarti ini juga 0

# [1] = positive review
# [0] = negative review

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string)


### Split Data

In [17]:
# pisah(split) menjadi data train dan data test
#masing-masing ada 25000 data

# Memisahkan review dan label untuk dataset train dan test
train_data, test_data = imdb['train'], imdb['test']

#ambil review  dari data train tanpa melihat label
#map() : membaca setiap (review, label) dari data
train_reviews = train_data.map(lambda review, label:review)
train_labels = train_data.map(lambda review, label:label)

test_reviews = test_data.map(lambda review, label:review)
test_labels = test_data.map(lambda review, label:label)

In [18]:
# Hyperparameters
VOCAB_SIZE = 10000
MAX_LENGTH = 120
EMBEDDING_DIM = 16
PADDING_TYPE = 'pre'
TRUNC_TYPE = 'post'

In [19]:
#buat layer
vectorize_layer=tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE)

vectorize_layer.adapt(train_reviews)


In [20]:
#fungsi padding : memastikan setiap urutan teks memiliki panjang yang sama
def padding_func(sequences):
    #batch:sekelompok data yang diproses bersamaan
    
    #membuat batch  
    #sequences.cardinality(): menghitung berapa banyak item yang ada dalam dataset sequences
    sequences=sequences.ragged_batch(batch_size=sequences.cardinality())
    
    #mengambil satu batch (berisi banyak elemen data) dari data?
    sequences = sequences.get_single_element()
    
    padded_sequences=tf.keras.utils.pad_sequences(sequences.numpy(), #ubah bentuk data agar bisa diolah
                                                  maxlen=120, #panjang maksimal urutan 120
                                                  truncating='post', #memotong urutan dari belakang jika terlalu panjang
                                                  padding='pre') #menambahkan padding di depan jika terlalu pendek
    
    padded_sequences=tf.data.Dataset.from_tensor_slices(padded_sequences) 
    #tadi kan data diubah jadi numpy, sekarang diubah jadi tensorflow lagi
    
    # Padding dilakukan setelah sequence dibentuk oleh TextVectorization
    return padded_sequences

In [21]:
#setelah ubah kalimat string menjadi integer, apply method padding untuk sama ratakan urutan

# Menerapkan vectorization dan padding
train_sequences = train_reviews.map(lambda text:vectorize_layer(text)).apply(padding_func)
test_sequences = test_reviews.map(lambda text:vectorize_layer(text)).apply(padding_func)

In [32]:
# View 2 training sequences and its labels
for example in train_sequences.take(2):
  print(example)
  print()

tf.Tensor(
[   0    0    0    0   11   14   34  412  384   18   90   28    1    8
   33 1322 3560   42  487    1  191   24   85  152   19   11  217  316
   28   65  240  214    8  489   54   65   85  112   96   22 5596   11
   93  642  743   11   18    7   34  394 9522  170 2464  408    2   88
 1216  137   66  144   51    2    1 7558   66  245   65 2870   16    1
 2860    1    1 1426 5050    3   40    1 1579   17 3560   14  158   19
    4 1216  891 8040    8    4   18   12   14 4059    5   99  146 1241
   10  237  704   12   48   24   93   39   11 7339  152   39 1322    1
   50  398   10   96 1155  851  141    9], shape=(120,), dtype=int32)

tf.Tensor(
[   0    0    0    0    0    0    0    0   10   26   75  617    6  776
 2355  299   95   19   11    7  604  662    6    4 2129    5  180  571
   63 1403  107 2410    3 3905   21    2    1    3  252   41 4781    4
  169  186   21   11 4259   10 1507 2355   80    2   20   14 1973    2
  114  943   14 1740 1300  594    3  356  180  446    6

## Preparing Data

In [22]:
# menggabungkan train_sequences, train_labels menjadi (input, label) dalam satu dataset
#seperti membuat 2 kolom
train_dataset_vectorized=tf.data.Dataset.zip(train_sequences, train_labels)

test_dataset_vectorized=tf.data.Dataset.zip(test_sequences, train_labels)


Preview data

In [23]:
# View 2 training sequences and its labels
for example in train_dataset_vectorized.take(2):
  print(example)
  print()

(<tf.Tensor: shape=(120,), dtype=int32, numpy=
array([   0,    0,    0,    0,   11,   14,   34,  412,  384,   18,   90,
         28,    1,    8,   33, 1322, 3560,   42,  487,    1,  191,   24,
         85,  152,   19,   11,  217,  316,   28,   65,  240,  214,    8,
        489,   54,   65,   85,  112,   96,   22, 5596,   11,   93,  642,
        743,   11,   18,    7,   34,  394, 9522,  170, 2464,  408,    2,
         88, 1216,  137,   66,  144,   51,    2,    1, 7558,   66,  245,
         65, 2870,   16,    1, 2860,    1,    1, 1426, 5050,    3,   40,
          1, 1579,   17, 3560,   14,  158,   19,    4, 1216,  891, 8040,
          8,    4,   18,   12,   14, 4059,    5,   99,  146, 1241,   10,
        237,  704,   12,   48,   24,   93,   39,   11, 7339,  152,   39,
       1322,    1,   50,  398,   10,   96, 1155,  851,  141,    9])>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)

(<tf.Tensor: shape=(120,), dtype=int32, numpy=
array([   0,    0,    0,    0,    0,    0,    0,    0,   10,

### Perbedaan Utama
- Bagian Pertama melakukan preprocessing untuk mengubah teks mentah menjadi angka dan menambahkan padding agar panjangnya sama.
- Bagian Kedua menggabungkan data yang sudah diproses ini (train_sequences) dengan labelnya (train_labels) dalam satu dataset.
  
#### Kesimpulan
Bagian Pertama adalah langkah preprocessing, sedangkan Bagian Kedua adalah langkah untuk menyusun dataset akhir ((input, label)) yang siap dilatih oleh model. Jika preprocessing tidak dilakukan terlebih dahulu, model tidak akan bisa membaca input teks.

In [ ]:
# Menyiapkan dataset untuk training
SHUFFLE_BUFFER_SIZE=1000
PREFETCH_BUFFER_SIZE = tf.data.AUTOTUNE
BATCH_SIZE=32

#Optimizer dataset untuk ditraining
train_dataset_final=(train_dataset_vectorized
                     .cache()
                     .shuffle(SHUFFLE_BUFFER_SIZE)
                     .prefetch(PREFETCH_BUFFER_SIZE)
                     .batch(BATCH_SIZE))

test_dataset_final=(test_dataset_vectorized
                     .cache()
                     .prefetch(PREFETCH_BUFFER_SIZE)
                     .batch(BATCH_SIZE))

In [25]:
model=tf.keras.Sequential([
    tf.keras.Input(shape=(120,)),
    tf.keras.layers.Embedding(VOCAB_SIZE, EMBEDDING_DIM),
    
    #jadikan array 1D
    #tf.keras.layers.Flatten() ,
    tf.keras.layers.GlobalAveragePooling1D(),
    
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

- flatten akan menghasilkan shape yang besar
- untuk alternatif, bisa menggunakan 
  tf.keras.layers.GlobalAveragePooling1D(),
  
  menghasilkan array 1D dengan shape lebih kecil yang lebih simple dan lebih cepat diproses.

Walaupun Flatten lebih lambat, tapi flatten lebih akurat

### Compile model

In [26]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy']
              )

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 120, 16)        │       160,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 16)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 6)              │           102 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │             7 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 160,109 (625.43 KB)

 Trainable params: 160,109 (625.43 KB)

 Non-trainable params: 0 (0.00 B)

### Train Model

In [27]:
EPOCHS=10
model.fit(train_dataset_final,
          epochs=EPOCHS,
          validation_data=test_dataset_final)

Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6633 - loss: 0.6390 - val_accuracy: 0.4987 - val_loss: 0.9539
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8442 - loss: 0.3735 - val_accuracy: 0.5022 - val_loss: 1.2554
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8760 - loss: 0.2981 - val_accuracy: 0.5003 - val_loss: 1.4329
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8946 - loss: 0.2579 - val_accuracy: 0.4985 - val_loss: 1.5577
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9111 - loss: 0.2291 - val_accuracy: 0.4980 - val_loss: 1.6641
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9206 - loss: 0.2088 - val_accuracy: 0.4988 - val_loss: 1.7778
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9285 - loss: 0.1916 - val_accuracy: 0.4975 - val_loss: 1.8786
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9356 - loss: 0.1795 - val_accuracy: 0.

### Menggunakan Flatten

In [28]:
model2=tf.keras.Sequential([
    tf.keras.Input(shape=(120,)),
    tf.keras.layers.Embedding(VOCAB_SIZE, EMBEDDING_DIM),
    
    #jadikan array 1D
    tf.keras.layers.Flatten() ,
    #tf.keras.layers.GlobalAveragePooling1D(),
    
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [29]:
model2.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy']
              )

EPOCHS=10
model2.fit(train_dataset_final,
          epochs=EPOCHS,
          validation_data=test_dataset_final)

Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6326 - loss: 0.6093 - val_accuracy: 0.4990 - val_loss: 1.2132
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8789 - loss: 0.2925 - val_accuracy: 0.4971 - val_loss: 1.6111
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9544 - loss: 0.1376 - val_accuracy: 0.4988 - val_loss: 2.0734
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9934 - loss: 0.0371 - val_accuracy: 0.4960 - val_loss: 2.5432
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9984 - loss: 0.0108 - val_accuracy: 0.4974 - val_loss: 2.9335
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9996 - loss: 0.0039 - val_accuracy: 0.4964 - val_loss: 3.1566
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 1.0000 - loss: 0.0016 - val_accuracy: 0.4980 - val_loss: 3.5009
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 1.0000 - loss: 5.4596e-04 - val_accuracy

## Visualisasi Word Embeddings

#### Embedding

 Lapisan embedding berfungsi untuk mengonversi kata-kata atau token yang diinput menjadi representasi vektor berukuran tetap (dimensi embedding).

In [30]:
#mengakses lapisan pertama (layers[0]) dari model, sebagai lapisan embedding?

# Mengambil bobot dari lapisan embedding
embedding_layer=model.layers[0]

#lapisan [0] : lapisan embedding
#lapisan [1] : lapisan dense
#lapisan [2] : lapisan output

#mengembil bobot dari lapisan embedding
embedding_weights =  embedding_layer.get_weights()[0]

print(embedding_weights.shape) #shape: (covab_size, embedding_dim)

(10000, 16)


Contoh

Jika vocab_size = 5000 dan embedding_dim = 64, maka ukuran dari embedding_weights akan menjadi (5000, 64), yang berarti terdapat 5000 kata dalam kosakata, masing-masing direpresentasikan oleh vektor embedding berdimensi 64.

### Plotting

In [ ]:
#Membuka file dan menyimpan data

#vecs.tsv: simpan vector embedding dari setiap kata
#bukan dan write(bisa menulis di file)
out_v=io.open('vecs.tsv', 'w', encoding='utf-8')

#meta.tsv: menyimpan kata-kata atau "metadata"-nya.
out_m=io.open('meta.tsv', 'w', encoding='utf-8')

#mengambil semua kata di vocabulary dari vectorize_layer
vocabulary = vectorize_layer.get_vocabulary()

#Menulis Kata dan Vektor Embedding ke File

#iterasi kata pertama sampai terakhir di vocabulary
#langsung 1 karena 0 itu padding
for word_num in range(1, len(vocabulary)):
   
    word_name = vocabulary[word_num] #Mengambil Nama Kata
    word_embedding = embedding_weights[word_num] #Mengambil Vektor Embedding Kata
    
    #Menulis ke File
    out_m.write(word_name + '\n') 
    out_v.write('\t'.join([str(x) for x in word_embedding]) + "\n")
    
out_v.close()
out_m.close()


- Mulai dari 1 agar tidak menuliskan "kata kosong" yang biasa ada di posisi 0 untuk padding.
- word_name berisi kata yang akan ditulis ke file meta.tsv
- word_embedding adalah array yang menyimpan representasi vektor dari word_name.
- Menulis word_name ke file meta.tsv di setiap baris, agar file ini berisi daftar kata.
- Menulis vektor word_embedding ke vecs.tsv. Setiap nilai dalam array word_embedding dipisahkan dengan \t (tab) agar vektor ini berbentuk tab-delimited pada setiap barisnya.

### Analogi
Anggap saja vocabulary ini seperti kamus di mana setiap kata punya definisi. Di sini, meta.tsv adalah daftar kata yang berfungsi seperti daftar isi buku, dan vecs.tsv adalah bagian yang memuat penjelasan atau deskripsi rinci (vektor embedding) dari setiap kata.

Contohnya, jika vocabulary berisi ["", "good", "bad", "excellent", "terrible"], maka:

- meta.tsv akan berisi:
  - good
  - bad
  - excellent
  - terrible
  
- vecs.tsv mungkin akan berisi sesuatu seperti:
  
  0.1    -0.4   0.6    ...   (embedding untuk "good")
  
  -0.3   0.8   -0.2    ...   (embedding untuk "bad")
 
  ...

Setiap kata memang memiliki satu token (atau indeks unik) dalam model, tetapi embedding adalah representasi numerik dari kata yang menangkap maknanya dalam bentuk vektor. Jadi, meskipun satu kata hanya memiliki satu token, embedding adalah array (vektor) dari beberapa nilai, yang semuanya bersama-sama menggambarkan fitur atau arti kata tersebut di dalam ruang multidimensi.

#### Mengapa Setiap Kata Memiliki Banyak Nilai Embedding?

Embedding dalam NLP bertujuan untuk menangkap makna atau konteks kata dalam ruang vektor yang kaya informasi. Setiap nilai dalam vektor embedding ini berkontribusi untuk merepresentasikan karakteristik tertentu dari kata tersebut. Misalnya, jika embedding berukuran 16 (Jumlah dimensinya), maka setiap kata akan memiliki vektor berisi 16 angka. Angka-angka ini tidak mewakili kata dalam bentuk satu nilai saja, tetapi sebagai kumpulan fitur-fitur (misalnya: asosiasi positif/negatif, kategori umum, atau hubungan dengan kata lain).

#### Cara Embedding Bekerja
Misalnya, jika kita punya kata “good,” dan embedding berukuran 16, model akan memberikan representasi yang tampak seperti ini:

[0.1, -0.2, 0.8, ..., -0.4]

Setiap angka dalam vektor ini mewakili aspek tertentu dari kata “good.” Vektor ini akan berbeda untuk kata lain, misalnya “bad,” karena model membedakan makna masing-masing kata berdasarkan pola hubungan mereka di dalam teks yang dilatihkan.

#### Analogi
Pikirkan embedding seperti menggambarkan karakter seseorang dalam berbagai dimensi (misalnya, kecerdasan, kebaikan, humor, kepercayaan, dll.). Seseorang mungkin memiliki nilai berbeda di masing-masing dimensi tersebut, dan ketika kita melihat keseluruhan vektor (semua dimensi), kita mendapatkan gambaran lengkap tentang karakter orang tersebut. Begitu juga, embedding sebuah kata memetakan kata dalam berbagai dimensi yang, secara keseluruhan, memberikan “makna” kata tersebut dalam konteks model.

Jadi, embedding bukan sekadar satu angka, tapi satu "potret" vektor multidimensi yang menggambarkan arti sebuah kata.